In [0]:
import yaml

with open('ingestion_config.yml', 'r') as file:
    config = yaml.safe_load(file)

databricks_config = config['databricks_config']
llm_config = config['llm_config']

catalog=databricks_config['catalog']
schema=databricks_config['schema']
bronze_table=databricks_config['bronze_table']

In [0]:
docs_bronze = spark.table(f"{catalog}.{schema}.{bronze_table}")

In [0]:
display(docs_bronze)

In [0]:
#extract document year and company name
docs_silver_with_year_company = docs_bronze.selectExpr("*","ai_extract(path, array('year', 'company')) AS extracted_data")

In [0]:
#extract document type
docs_silver_with_doc_type = docs_silver_with_year_company.selectExpr(
    "*",
    """
    ai_classify(path, ARRAY('10k', '8k', '10q', 'Earnings Report')) AS document_type
    """,
)

In [0]:
docs_silver_with_doc_type.write.saveAsTable("doan.alphaledger.sec_docs_silver")

In [0]:
display(spark.table("doan.alphaledger.sec_docs_silver"))